## Scraping BT for Novo Nordisk

In [28]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select

from bs4 import BeautifulSoup
import time
import re
import requests
import pandas as pd

In [29]:
#Get the response
#response=requests.get('hhttps://www.berlingske.dk', headers={'name': 'Lea Lund Jeppesen', 'e-mail':'snk956@alumni.ku.dk'})

# Set up the driver and navigate to the URL
url = 'https://www.berlingske.dk'
driver = webdriver.Chrome()  # Ensure you have the ChromeDriver set up correctly
driver.get(url)

# Find and click on the "Allow Cookies" button
allow_cookies_button = driver.find_element(By.CSS_SELECTOR, "#CybotCookiebotDialogBodyButtonDecline")
allow_cookies_button.click()

#Find and click the menu button
menu_data_id = "menuToggler"
menu_element = driver.find_element(By.CSS_SELECTOR, f"[data-id='{menu_data_id}']")
menu_element.click()

# Locate the search input element by its ID
search_input_id = "header-search"
search_input = driver.find_element(By.ID, search_input_id)

# Perform a search action in the search input
search_input.send_keys("Novo Nordisk")
search_input.submit()

In [30]:
#Each page contains 10 articles. We want to collect 1000 so we click the load more button 100 times
for i in range(0,1000):
    load_button=driver.find_element(By.XPATH, '/html/body/div[2]/div[4]/div[2]/div/div/div/div[2]/button')
    load_button.click()
    i=i+1
    time.sleep(0.1)

In [31]:
# Get the page source
page_source = driver.page_source

# Use Beautiful Soup to parse the HTML
soup = BeautifulSoup(page_source, "html.parser")


# Find all article containers with the specified class
article_containers = soup.find_all('div', class_='teaser teaser--border-bottom teaser--search d-flex flex-row w-100')

# Lists to hold extracted data
titles = []
dates = []

# Loop through the article containers and extract title and date
for container in article_containers:
    title = container.find('a', class_='teaser__title-link').text.strip()
    date = container.find('div', class_='teaser__date').text.strip()
    
    titles.append(title)
    dates.append(date)

# Convert the lists to a pandas DataFrame
df = pd.DataFrame({
    'Title': titles,
    'Date': dates})

df


,Title,Date
0,Business-overblik: Lauritz-manøvre skjulte dun...,11. august 2023
1,Novo vokser og vokser. Er nu mere værd end Por...,10. august 2023
2,Lars Fruergaard affejer kritik af mangel på di...,10. august 2023
3,Business-update: »En narkobande af Mærsk-ansat...,10. august 2023
4,Novo Nordisk kæmper med at følge den store eft...,10. august 2023
...,...,...
2676,Mærsk-chef: Vi tjener stadig for lidt,9. februar 2018
2677,Mærsk-topchef Der er god vækst på tværs af hel...,9. februar 2018
2678,Kåre Schultz skuffer i Tel Aviv - Tevas aktie ...,8. februar 2018
2679,Firmabilisterne vælger premium - og automatgear,8. februar 2018
